# 04 - Load to Database (Carregar para Banco de Dados)

Carregar dados das camadas Silver e Gold para banco de dados SQLite para análise e consultas.


In [2]:
import pandas as pd
import sqlite3
from datetime import datetime
import os


## 1. Conectar ao Banco de Dados


In [ ]:
# Conectar ao banco SQLite
os.makedirs('data', exist_ok=True)
conn = sqlite3.connect('data/pipeline.db')

print(" Conexão com banco de dados estabelecida")


✓ Conexão com banco de dados estabelecida


## 2. Carregar Dados da Camada Silver


In [4]:
# Carregar dados limpos (Silver layer)
df_limpo = pd.read_csv('data/silver/dados_limpos.csv')

# Converter InvoiceDate para datetime
df_limpo['InvoiceDate'] = pd.to_datetime(df_limpo['InvoiceDate'])

# Salvar em tabela principal
df_limpo.to_sql('vendas', conn, if_exists='replace', index=False)

print(f"✓ Tabela 'vendas' criada: {len(df_limpo):,} registros")
print(f"✓ Colunas: {df_limpo.shape[1]}")
df_limpo.head()


✓ Tabela 'vendas' criada: 536,639 registros
✓ Colunas: 13


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,data_ingestao,fonte_arquivo,eh_devolucao,valor_total,data_processamento
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2025-10-31 19:38:51.638174,data.csv,False,15.30,2025-10-31 19:40:08.366383
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2025-10-31 19:38:51.638174,data.csv,False,20.34,2025-10-31 19:40:08.366383
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2025-10-31 19:38:51.638174,data.csv,False,22.00,2025-10-31 19:40:08.366383
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2025-10-31 19:38:51.638174,data.csv,False,20.34,2025-10-31 19:40:08.366383
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2025-10-31 19:38:51.638174,data.csv,False,20.34,2025-10-31 19:40:08.366383


## 3. Carregar Dados da Camada Gold


In [ ]:
# Carregar métricas diárias
metricas_diarias = pd.read_csv('data/gold/metricas_diarias.csv')
metricas_diarias['data'] = pd.to_datetime(metricas_diarias['data'])
metricas_diarias.to_sql('metricas_diarias', conn, if_exists='replace', index=False)
print(f" Tabela 'metricas_diarias': {len(metricas_diarias)} registros")


✓ Tabela 'metricas_diarias': 305 registros


In [ ]:
# Carregar análise de clientes RFM
analise_clientes = pd.read_csv('data/gold/analise_clientes.csv')
analise_clientes['ultima_compra'] = pd.to_datetime(analise_clientes['ultima_compra'])
analise_clientes.to_sql('analise_clientes', conn, if_exists='replace', index=False)
print(f"Tabela 'analise_clientes': {len(analise_clientes)} registros")


✓ Tabela 'analise_clientes': 4373 registros


In [ ]:
# Carregar desempenho de produtos
desempenho_produtos = pd.read_csv('data/gold/desempenho_produtos.csv')
desempenho_produtos.to_sql('desempenho_produtos', conn, if_exists='replace', index=False)
print(f" Tabela 'desempenho_produtos': {len(desempenho_produtos)} registros")


✓ Tabela 'desempenho_produtos': 5747 registros


In [ ]:
# Carregar análise de países
analise_paises = pd.read_csv('data/gold/analise_paises.csv')
analise_paises.to_sql('analise_paises', conn, if_exists='replace', index=False)
print(f" Tabela 'analise_paises': {len(analise_paises)} registros")


✓ Tabela 'analise_paises': 38 registros


In [ ]:
# Carregar métricas mensais
metricas_mensais = pd.read_csv('data/gold/metricas_mensais.csv')
metricas_mensais.to_sql('metricas_mensais', conn, if_exists='replace', index=False)
print(f" Tabela 'metricas_mensais': {len(metricas_mensais)} registros")


✓ Tabela 'metricas_mensais': 13 registros


In [ ]:
# Carregar análise de devoluções
if os.path.exists('data/gold/analise_devolucoes.csv'):
    analise_devolucoes = pd.read_csv('data/gold/analise_devolucoes.csv')
    analise_devolucoes.to_sql('analise_devolucoes', conn, if_exists='replace', index=False)
    print(f" Tabela 'analise_devolucoes': {len(analise_devolucoes)} registros")
else:
    print(" Arquivo de devoluções não encontrado")


✓ Tabela 'analise_devolucoes': 3181 registros


## 4. Criar Tabelas Relacionadas


In [ ]:
# Tabela de clientes únicos
clientes = df_limpo[['CustomerID', 'Country']].drop_duplicates(subset=['CustomerID'])
clientes.columns = ['id_cliente', 'pais']
clientes.to_sql('clientes', conn, if_exists='replace', index=False)
print(f" Tabela 'clientes': {len(clientes)} registros")


✓ Tabela 'clientes': 4373 registros


In [ ]:
# Tabela de produtos únicos
produtos = df_limpo[['StockCode', 'Description', 'UnitPrice']].drop_duplicates(subset=['StockCode'])
produtos.columns = ['codigo_produto', 'nome_produto', 'preco_unitario']
produtos.to_sql('produtos', conn, if_exists='replace', index=False)
print(f" Tabela 'produtos': {len(produtos)} registros")


✓ Tabela 'produtos': 4070 registros


## 5. Verificar Tabelas Criadas


In [ ]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = cursor.fetchall()


print("TABELAS NO BANCO DE DADOS")

for tabela in tabelas:
    cursor.execute(f"SELECT COUNT(*) FROM {tabela[0]};")
    count = cursor.fetchone()[0]
    print(f" {tabela[0]}: {count:,} registros")




TABELAS NO BANCO DE DADOS
✓ vendas: 536,639 registros
✓ metricas_diarias: 305 registros
✓ analise_clientes: 4,373 registros
✓ desempenho_produtos: 5,747 registros
✓ analise_paises: 38 registros
✓ metricas_mensais: 13 registros
✓ analise_devolucoes: 3,181 registros
✓ clientes: 4,373 registros
✓ produtos: 4,070 registros


## 6. Fechar Conexão


In [ ]:
conn.close()
print("\n Dados carregados com sucesso no banco de dados!")
print(f" Arquivo: data/pipeline.db")



✓ Dados carregados com sucesso no banco de dados!
✓ Arquivo: data/pipeline.db
